<h1><center>ЛОГИРОВАНИЕ В MLFLOW НА ПРАКТИКЕ</h1></center>

MLflow Tracking Server запущен (с помощью команды sh run_mlflow_server.sh) — можно начинать что-то туда логировать.<br>Перед этим стоит достать ваш датафрейм из базы данных Postgres. Сделать это можно при помощи кода, рассмотренного ранее: 

In [ ]:
import pandas as pd
import psycopg
import mlflow
import os

connection = {"sslmode": "require", "target_session_attrs": "read-write"}
postgres_credentials = { "host": "rc1b-uh7kdmcx67eomesf.mdb.yandexcloud.net", 
                         "port": "6432",
                         "dbname": "playground_mle_20250529_05fed48463",
                         "user": "mle_20250529_05fed48463",
                         "password": "0c567edd8ad8472e87d5c85cc4d664e4" }
assert all([var_value != "" for var_value in list(postgres_credentials.values())])

connection.update(postgres_credentials)

# определим название таблицы, в которой хранятся наши данные.
TABLE_NAME = "clean_users_churn"

# эта конструкция создаёт контекстное управление для соединения с базой данных 
# оператор with гарантирует, что соединение будет корректно закрыто после выполнения всех операций 
# закрыто оно будет даже в случае ошибки, чтобы не допустить "утечку памяти"
with psycopg.connect(**connection) as conn:

# создаёт объект курсора для выполнения запросов к базе данных
# с помощью метода execute() выполняется SQL-запрос для выборки данных из таблицы TABLE_NAME
    with conn.cursor() as cur:
        cur.execute(f"SELECT * FROM {TABLE_NAME}")
                
                # извлекаем все строки, полученные в результате выполнения запроса
        data = cur.fetchall()

                # получает список имён столбцов из объекта курсора
        columns = [col[0] for col in cur.description]

# создаёт объект DataFrame из полученных данных и имён столбцов. 
# это позволяет удобно работать с данными в Python, используя библиотеку Pandas.
df = pd.DataFrame(data, columns=columns) 

Ваша цель - создать текстовый файл с названием columns_sol.txt и сохранить в него названия колонок датафрейма, используя , как разделитель.

In [3]:
# Получаем список названий колонок из датафрейма df
columns = df.columns

# 1. Название колонок вашего датафрейма запишите в текстовый файл
with open("columns_sol.txt", "w", encoding="utf-8") as fio:
    fio.write(",".join(columns))

Посчитайте различные статистики (минимальные, максимальные, средние и медианы) по вашему датафрейму для определённых колонок (count_columns) и запишите результат в Python-словарь. Для подсчёта воспользуйтесь функцией value_counts. Это нужно, чтобы понимать, на каких данных была обучена ваша модель.

In [ ]:
counts_columns = ["type", "paperless_billing", "internet_service", "online_security", "online_backup", "device_protection",
    "tech_support", "streaming_tv", "streaming_movies", "gender", "senior_citizen", "partner", "dependents",
    "multiple_lines", "target"]

stats = {}

for col in counts_columns:
    # посчитайте уникальные значения для колонок, где немного уникальных значений (переменная counts_columns)
    column_stat = df[col].value_counts().to_dict()
    column_stat = {f"{col}_{key}": value for key, value in column_stat.items()}

    # обновите словарь stats
    stats.update(column_stat)

stats["data_length"] = df.shape[0]
stats["monthly_charges_min"] = df["monthly_charges"].min()
stats["monthly_charges_max"] = df["monthly_charges"].max() # посчитайте максимальное значение в колонке
stats["monthly_charges_mean"] = df["monthly_charges"].mean() # посчитайте среднее значение в колонке
stats["monthly_charges_median"] = df["monthly_charges"].median() # посчитайте медианное значение в колонке
stats["total_charges_min"] = df["total_charges"].min() # посчитайте минимальное значение в колонке
stats["total_charges_max"] = df["total_charges"].max() # посчитайте максимальное значение в колонке
stats["total_charges_mean"] = df["total_charges"].mean() # посчитайте среднее значение в колонке
stats["total_charges_median"] = df["total_charges"].median() # посчитайте медианное значение в колонке
stats["unique_customers_number"] = df["customer_id"].nunique() # посчитайте кол-во уникальных id
stats["end_date_nan"] = df["end_date"].isna().sum() # посчитайте кол-во пустых строк в колонке

Если в проекте есть такая возможность, а размер вашего датасета не превышает несколько гигабайт, то можно сохранить и сам датафрейм. Например, в формате CSV: 

In [5]:
df.to_csv('users_churn.csv',index=False)

Ваша задача:
- Залогируйте ваши первые артефакты, создав новый эксперимент. В качестве директории для артефактов используйте dataframe.
- Проверьте, что ваш запуск прошёл успешно.
- Удалите файл с колонками и сам датафрейм, чтобы они не занимали место на диске на локальной машине.

In [10]:
# задаём название эксперимента и имя запуска для логирования в MLflow

EXPERIMENT_NAME = "churn_fio_2"
RUN_NAME = "data_check_2"

# создаём новый эксперимент в MLflow с указанным названием 
# если эксперимент с таким именем уже существует, 
# MLflow возвращает идентификатор существующего эксперимента
experiment_id = mlflow.create_experiment(EXPERIMENT_NAME) if not mlflow.get_experiment_by_name(EXPERIMENT_NAME) else mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id

with mlflow.start_run(run_name=RUN_NAME, experiment_id=experiment_id) as run:
    mlflow.set_tag("mlflow.runName", RUN_NAME) 

    # получаем уникальный идентификатор запуска эксперимента
    run_id = run.info.run_id
    
    # логируем метрики эксперимента
    # предполагается, что переменная stats содержит словарь с метриками,
    # объявлять переменную stats не надо,
    # где ключи — это названия метрик, а значения — числовые значения метрик
    mlflow.log_metrics(stats)
    
    # логируем файлы как артефакты эксперимента — 'columns.txt' и 'users_churn.csv'
    mlflow.log_artifact("columns_sol.txt",'dataframe')
    mlflow.log_artifact("users_churn.csv",'dataframe') 


experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
# получаем данные о запуске эксперимента по его уникальному идентификатору
run = mlflow.get_run(run_id)


# проверяем, что статус запуска эксперимента изменён на 'FINISHED'
# это утверждение (assert) можно использовать для автоматической проверки того, 
# что эксперимент был завершён успешно
assert run.info.status == "FINISHED"

# удаляем файлы 'columns.txt' и 'users_churn.csv' из файловой системы,
# чтобы очистить рабочую среду после логирования артефактов
os.remove('columns_sol.txt')
os.remove('users_churn.csv') 